In [812]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
from datetime import datetime
import requests
from PyPDF2 import PdfReader
from io import BytesIO
import pandas as pd

In [9]:
# viewing settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 50)

#pd.reset_option("all")

In [ ]:
# ideas/to do

preferred PM/leader approval - leadership approval historic? 
economic optimism
leadership soccer styke radar charts
blue wall vs red wall

## Voting Intention

In [1098]:
# scrape polling data from Pollbase

pollbase_link = 'https://www.markpack.org.uk/155623/voting-intention-opinion-poll-scorecard/'

link = []

headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
pageTree = requests.get(pollbase_link, headers=headers)
pageSoup = BeautifulSoup(pageTree.content, 'html.parser')
pollbase_poll_url = pageSoup.find_all("td", {"class": "has-text-align-center"})
link.extend(pollbase_poll_url)

link = [str(item) for item in link]

# extract relevant data from html code and format so that it is ready for input into dataframe
data = []
for i in link:
    soup = BeautifulSoup(i, 'html.parser')
    extracted_text = soup.get_text()
    data.append(extracted_text)
    
sublists = [] # grouping every 8 items (8 variables per row)
max_items_per_sublist = 8
for i in range(0, len(data), max_items_per_sublist):
    sublist = data[i:i + max_items_per_sublist]
    sublists.append(sublist)
    
sublists = sublists[:-3] # delete 2015/17/19 result

def remove_patterns_and_simplify_date(item):
    # Remove patterns like (+2), (-1), and (nc)
    cleaned_item = re.sub(r'\(\+\d+\)|\(-\d+\)|\(nc\)', '', item)
    
    # Extract the simplified date and add year (e.g., '22-25/9' becomes '22/9/2023')
    date_match = re.search(r'(\d{1,2})-(\d{1,2})/(\d{1,2})', cleaned_item)
    if date_match:
        day_part = date_match.group(1)
        month_part = date_match.group(3)
        cleaned_item = f"{day_part}/{month_part}/2024"
    else:
        # 24/09 becomes 24/09/2023
        date_match = re.search(r'(\d{1,2})/(\d{1,2})', cleaned_item)
        if date_match:
            day_part = date_match.group(1)
            month_part = date_match.group(2)
            cleaned_item = f"{day_part}/{month_part}/2024"
    
    return cleaned_item

# Apply the function to each item in the list of lists
cleaned_data_list_of_lists = [[remove_patterns_and_simplify_date(item) for item in sublist] for sublist in sublists]

# Create a pandas DataFrame with cleaned data, convert %'s to floats and make date column a date

columns = ["Polling Company", "Con", "Lab", "Lib Dem", "Green", "Reform",'Con lead', "Date"]
df_latest = pd.DataFrame(cleaned_data_list_of_lists, columns=columns)
df_latest.iloc[:, 1:7] = df_latest.iloc[:, 1:7].apply(lambda x: pd.to_numeric(x.str.rstrip('%'), errors='coerce'))
df_latest['Date'] = pd.to_datetime(df_latest['Date'], format='%d/%m/%Y') # make datetime object
df_latest['Date'] = df_latest['Date'].dt.strftime('%d/%m/%Y') # convert to standard date/time formatting, ready to merge datasets

# create a second dataframe with historical polling data (currently up to Q4 2023)

df_historical = pd.read_excel('/Users/james1/Documents/Documents/global counsel/Pollbase Q4 2023.xlsx')
df_historical['Date'] = df_historical['Date'].dt.strftime('%d/%m/%Y')


# add dataframes to create overall dataset, reset date column to datetime object with standardised formatting

df_final = pd.concat([df_latest, df_historical], axis=0)
df_final['Date'] = pd.to_datetime(df_final['Date'], format='%d/%m/%Y')
df_final['Date'] = df_final['Date'].dt.strftime('%d/%m/%Y')
df_final.iloc[:, 1:7] /= 100

In [1108]:
# code to execute each time i want to update data

pollbase_link = 'https://www.markpack.org.uk/155623/voting-intention-opinion-poll-scorecard/'

link = []

headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
pageTree = requests.get(pollbase_link, headers=headers)
pageSoup = BeautifulSoup(pageTree.content, 'html.parser')
pollbase_poll_url = pageSoup.find_all("td", {"class": "has-text-align-center"})
link.extend(pollbase_poll_url)

link = [str(item) for item in link]

# extract relevant data from html code and format so that it is ready for input into dataframe
data = []
for i in link:
    soup = BeautifulSoup(i, 'html.parser')
    extracted_text = soup.get_text()
    data.append(extracted_text)
    
sublists = [] # grouping every 8 items (8 variables per row)
max_items_per_sublist = 8
for i in range(0, len(data), max_items_per_sublist):
    sublist = data[i:i + max_items_per_sublist]
    sublists.append(sublist)
    
sublists = sublists[:-3] # delete 2015/17/19 result

cleaned_data_list_of_lists = [[remove_patterns_and_simplify_date(item) for item in sublist] for sublist in sublists]


columns = ["Polling Company", "Con", "Lab", "Lib Dem", "Green", "Reform",'Con lead', "Date"]
df_latest = pd.DataFrame(cleaned_data_list_of_lists, columns=columns)
df_latest.iloc[:, 1:7] = df_latest.iloc[:, 1:7].apply(lambda x: pd.to_numeric(x.str.rstrip('%'), errors='coerce'))
df_latest['Date'] = pd.to_datetime(df_latest['Date'], format = '%d/%m/%Y') # make datetime object
df_latest['Date'] = df_latest['Date'].dt.strftime('%d/%m/%Y') # convert to standard date/time formatting, ready to merge datasets
df_latest.iloc[:, 1:7] /= 100 # divide new data by 100 so in same unit as final df

# add the new latest df to the df_final that combines historical data with the 'old latest' data

df_final = pd.concat([df_latest, df_final], axis=0, ignore_index=True)

# sort dataframe by date

df_final['Date'] = pd.to_datetime(df_final['Date'],format = '%d/%m/%Y')
df_final = df_final.sort_values(by='Date')
df_final['Date'] = df_final['Date'].dt.strftime('%d/%m/%Y')

# get rid of any duplicates (polls still on Pack's latest data scrape but that were scraped last time)

df_final = df_final.drop_duplicates(ignore_index = True)

df_final.to_excel('votingintention.xlsx', index=False) 

In [ ]:
# Plot VI tracker graph 

fig, ax = plt.subplots()
fig, ax = plt.subplots(figsize=(40, 24))

# plot different lines
ax.plot(df3['Date'],df3['Con'], label='Con', color='blue')
ax.plot(df3['Date'],df3['Lab'], label='Lab', color='red')
ax.plot(df3['Date'],df3['Lib Dem'], label='LD', color='#FFA200')
ax.plot(df3['Date'],df3['Green'], label='Green', color='green')
ax.plot(df3['Date'],df3['Reform'], label='Reform', color='turquoise')

# formatting

def percent_formatter(x, pos):
    return f'{x*1:.0f}%'
ax.yaxis.set_major_formatter(FuncFormatter(percent_formatter))
ax.yaxis.set_tick_params(labelsize=30)
#ax.set_xticks(df3['Date'])
#ax.set_xticklabels([date.strftime('%b %Y') for date in df3['Date']])
ax.xaxis.set_tick_params(labelsize=30)
ax.legend(loc='upper center', bbox_to_anchor=(0.25, 0.45), ncol=3, fontsize = 30) 

In [ ]:
# show bar chart of Conservative lead 

colors = ['blue' if val > 0 else 'red' for val in df3['Con lead'][:15]]

# Create a figure and axis

fig, ax = plt.subplots()
fig, ax = plt.subplots(figsize=(20, 12))

# Create the bar chart
bars = ax.bar(df3['Date'][:15], df3['Con lead'][:15], color=colors)

# Set labels and title

date_format = mdates.DateFormatter('%d-%b-%Y')
ax.xaxis.set_major_formatter(date_format)

# Add labels to the bars
for bar, val in zip(bars, df3['Con lead'][:15]):
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height(),
            str(val), ha='center', va='top', fontsize=15)

# Show the plot
plt.show()

## Ipsos Issues

In [1287]:
def get_issues_data_from_pdf(date_of_publish_str, date_str):
    # Define the convert_date_format function
    def convert_date_format(date_of_publish_str):
        # Parse the month and year from the input date string
        month_str = date_of_publish_str[:3]
        year_str = '20' + date_of_publish_str[-2:]

        # Convert the month abbreviation to its numerical representation
        month_dict = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06', 
                      'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}
        month_numeric = month_dict[month_str]

        # Format the date into 'YYYY-MM' format
        formatted_date = f'{year_str}-{month_numeric}'
        return formatted_date
    
    def convert_date_str_to_mmm_yy(date_str): #1: Jan24 -> jan24
        # Parse the month and year from the input date string
        month_str = date_str[:3]
        year_str = date_str[-2:]

        # Convert the month abbreviation to its full name
        month_dict_lower = {'Jan': 'jan', 'Feb': 'feb', 'Mar': 'mar', 'Apr': 'apr', 'May': 'may', 'Jun': 'jun', 
                  'Jul': 'jul', 'Aug': 'aug', 'Sep': 'sep', 'Oct': 'october', 'Nov': 'nov', 'Dec': 'dec'}
        month_lower = month_dict_lower[month_str]

        # Format the date into 'mmm-year' format
        formatted_date_lower = f'{month_lower}{year_str}'
        return formatted_date_lower
    
    def convert_date_str_to_month_yy(date_str): #2 Jan24 -> january24
        # Parse the month and year from the input date string
        month_str = date_str[:3]
        year_str = date_str[-2:]

        # Convert the month abbreviation to its full name
        month_dict_lower = {'Jan': 'january', 'Feb': 'february', 'Mar': 'march', 'Apr': 'april', 'May': 'may', 'Jun': 'june', 
                  'Jul': 'july', 'Aug': 'august', 'Sep': 'september', 'Oct': 'oct', 'Nov': 'november', 'Dec': 'december'}
        month_full_name_lower = month_dict_lower[month_str]

        # Format the date into 'month-year' format
        formatted_date_lower = f'{month_full_name_lower}{year_str}'
        return formatted_date_lower
    
    def convert_date_str_to_Month_yy(date_str): #3 Jan24 -> January24
        # Parse the month and year from the input date string
        month_str = date_str[:3]
        year_str = date_str[-2:]

        # Convert the month abbreviation to its full name
        month_dict_upper = {'Jan': 'January', 'Feb': 'February', 'Mar': 'March', 'Apr': 'April', 'May': 'May', 'Jun': 'June', 
                  'Jul': 'July', 'Aug': 'August', 'Sep': 'September', 'Oct': 'October', 'Nov': 'November', 'Dec': 'December'}
        month_full_name_upper = month_dict_upper[month_str]

        # Format the date into 'Month-Year' format
        formatted_date_upper = f'{month_full_name_upper}{year_str}'
        return formatted_date_upper
    
    def convert_date_str_to_mmm_year(date_str): #4 Jan24 -> jan2024
        # Parse the month and year from the input date string
        month_str = date_str[:3]
        year_str = '20' + date_str[-2:]

        # Convert the month abbreviation to its full name
        month_dict_lower = {'Jan': 'jan', 'Feb': 'feb', 'Mar': 'mar', 'Apr': 'apr', 'May': 'may', 'Jun': 'jun', 
                  'Jul': 'jul', 'Aug': 'aug', 'Sep': 'sep', 'Oct': 'october', 'Nov': 'nov', 'Dec': 'dec'}
        month_lower = month_dict_lower[month_str]

        # Format the date into 'month-year' format
        formatted_date_lower = f'{month_lower}{year_str}'
        return formatted_date_lower
    
    def convert_date_str_to_Mmm_year(date_str): #5 Jan24 -> Jan2024
        # Parse the month and year from the input date string
        month_str = date_str[:3]
        year_str = '20' + date_str[-2:]

        # Convert the month abbreviation to its full name
        month_dict_upper = {'Jan': 'Jan', 'Feb': 'Feb', 'Mar': 'Mar', 'Apr': 'Apr', 'May': 'May', 'Jun': 'Jun', 
                  'Jul': 'Jul', 'Aug': 'Aug', 'Sep': 'Sep', 'Oct': 'Oct', 'Nov': 'Nov', 'Dec': 'Dec'}
        month_upper = month_dict_upper[month_str]

        # Format the date into 'Month-Year' format
        formatted_date_upper = f'{month_upper}{year_str}'
        return formatted_date_upper

    def convert_date_str_to_month_year(date_str): #6 Jan24 -> january2024
        # Parse the month and year from the input date string
        month_str = date_str[:3]
        year_str = '20' + date_str[-2:]

        # Convert the month abbreviation to its full name
        month_dict_lower = {'Jan': 'january', 'Feb': 'february', 'Mar': 'march', 'Apr': 'april', 'May': 'may', 'Jun': 'june', 
                  'Jul': 'july', 'Aug': 'august', 'Sep': 'september', 'Oct': 'october', 'Nov': 'november', 'Dec': 'december'}
        month_full_name_lower = month_dict_lower[month_str]

        # Format the date into 'month-year' format
        formatted_date_lower = f'{month_full_name_lower}{year_str}'
        return formatted_date_lower
        
    def convert_date_str_to_Month_year(date_str): #7 Jan24 -> January2024
        # Parse the month and year from the input date string
        month_str = date_str[:3]
        year_str = '20' + date_str[-2:]

        # Convert the month abbreviation to its full name
        month_dict_upper = {'Jan': 'January', 'Feb': 'February', 'Mar': 'March', 'Apr': 'April', 'May': 'May', 'Jun': 'June', 
                  'Jul': 'July', 'Aug': 'August', 'Sep': 'September', 'Oct': 'October', 'Nov': 'November', 'Dec': 'December'}
        month_full_name_upper = month_dict_upper[month_str]

        # Format the date into 'Month-Year' format
        formatted_date_upper = f'{month_full_name_upper}{year_str}'
        return formatted_date_upper
    
    
    def convert_date_str_to_month_dash_year (date_str): #8 Jan24 -> january-2024
        # Parse the month and year from the input date string
        month_str = date_str[:3]
        year_str = '20' + date_str[-2:]

        # Convert the month abbreviation to its full name
        month_dict_lower = {'Jan': 'january', 'Feb': 'february', 'Mar': 'march', 'Apr': 'april', 'May': 'may', 'Jun': 'june', 
                  'Jul': 'july', 'Aug': 'august', 'Sep': 'september', 'Oct': 'october', 'Nov': 'november', 'Dec': 'december'}
        month_full_name_lower = month_dict_lower[month_str]

        # Format the date into 'month-year' format
        formatted_date_lower = f'{month_full_name_lower}-{year_str}'
        return formatted_date_lower
    
    def convert_date_str_to_mmm_dash_year (date_str): #9 Jan24 -> jan-2024
        # Parse the month and year from the input date string
        month_str = date_str[:3]
        year_str = '20' + date_str[-2:]

        # Convert the month abbreviation to its full name
        month_dict_lower = {'Jan': 'jan', 'Feb': 'feb', 'Mar': 'mar', 'Apr': 'apr', 'May': 'may', 'Jun': 'jun', 
                  'Jul': 'jul', 'Aug': 'aug', 'Sep': 'sep', 'Oct': 'oct', 'Nov': 'nov', 'Dec': 'dec'}
        month_lower = month_dict_lower[month_str]

        # Format the date into 'month-year' format
        formatted_date_lower = f'{month_lower}-{year_str}'
        return formatted_date_lower
    
    # Convert date_of publish_str to 'YYYY-MM' format
    formatted_date_of_publish_str = convert_date_format(date_of_publish_str)
    
    # Convert date_str to MM/YYYY format for the date column of data frame
    date_obj = datetime.strptime(date_str, '%b%y')
    formatted_date_str = date_obj.strftime('%m/%Y')
    
    # Convert date_str into all different formats for scraping 
    date_str_lower_year = convert_date_str_to_mmm_year(date_str) # Jan24 -> jan2024
    date_str_lower_full_month_year = convert_date_str_to_month_year(date_str) #Jan24 -> january2024
    date_str_full_month_year = convert_date_str_to_Month_year(date_str) #Jan24 -> January2024
    date_str_upper_year = convert_date_str_to_Mmm_year(date_str) #Jan24 -> Jan2024
    date_str_lower_yy = convert_date_str_to_mmm_yy(date_str) #Jan24 -> jan24
    date_str_lower_full_month_yy = convert_date_str_to_month_yy(date_str) #Jan24 -> january24
    date_str_full_month_yy = convert_date_str_to_Month_yy(date_str) #Jan24 -> January24
    date_str_lower_full_month_dash_year = convert_date_str_to_month_dash_year(date_str) #Jan24 -> january-2024
    date_str_lower_dash_year = convert_date_str_to_mmm_dash_year(date_str) #Jan24 -> jan-2024

    # List of possible URL formats
    url_formats = [
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str}_Issues%20Index_topline_PUBLIC.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str}_Issues%20Index_topline_PUBLIC_0.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_lower_year}_Issues%20Index_topline_PUBLIC.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_lower_full_month_year}_Issues%20Index_topline_PUBLIC.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_full_month_year}_Issues%20Index_topline_PUBLIC.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_upper_year}_Issues%20Index_topline_PUBLIC.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_lower_yy}_Issues%20Index_topline_PUBLIC.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_lower_full_month_yy}_Issues%20Index_topline_PUBLIC.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_full_month_yy}_Issues%20Index_topline_PUBLIC.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/ipsos-issues-index-{date_str_lower_full_month_dash_year}-topline.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/ipsos-issues-index-{date_str_lower_full_month_dash_year}-topline-PUBLIC.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/ipsos-issues-index-{date_str_lower_dash_year}-topline.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/ipsos-issues-index-{date_str_lower_dash_year}-topline-PUBLIC.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/ipsos-issues-index-topline-{date_str_lower_dash_year}.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/ipsos-issues-index-topline-{date_str_lower_full_month_dash_year}.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/issues-index-{date_str_lower_full_month_dash_year}-topline.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/issues-index-{date_str_lower_dash_year}-topline.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/issues-index-{date_str_lower_full_month_dash_year}-topline-PUBLIC.pdf',  
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/issues-index-{date_str_lower_dash_year}-topline-PUBLIC.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/issues-index-topline-{date_str_lower_dash_year}.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/issues-index-topline-{date_str_lower_full_month_dash_year}.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str}_issues_index_topline_public.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str}_issues_index_topline.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_lower_year}_issues_index_topline_public.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_lower_full_month_year}_issues_index_topline_public.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_full_month_year}_issues_index_topline_public.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_upper_year}_issues_index_topline_public.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_lower_yy}_issues_index_topline_public.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_lower_full_month_yy}_issues_index_topline_public.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_full_month_yy}_issues_index_topline_public.pdf'
    ]
        
    
    # Attempt to retrieve the PDF content from each URL
    for pdf_url in url_formats:
        response = requests.get(pdf_url)
        if response.status_code == 200:
            pdf_file = PyPDF2.PdfReader(BytesIO(response.content))
            break  # Break out of the loop if PDF content is successfully retrieved
    
    # If PDF content couldn't be retrieved, handle the error or return None
    else:
        print(response.status_code, pdf_url)
        return None
    
    # Get the PDF content
    #response = requests.get(pdf_url)
    #pdf_file = PyPDF2.PdfReader(BytesIO(response.content))
    
     # Extract text from each page of the PDF
    for page_number in range(len(pdf_file.pages)):
        page_text = pdf_file.pages[page_number].extract_text()
    rows = [line.split() for line in page_text.split('\n') if line.strip()]
    rows = rows[10:]
    
    # Rearrange each sublist
    def rearrange_sublist(sublist):
        variable_names = ' '.join(sublist[:-2])
        first_value = sublist[-2] if len(sublist) >= 2 else ''
        second_value = sublist[-1] if len(sublist) >= 1 else ''
        return [variable_names, first_value, second_value]

    rearranged_data = [rearrange_sublist(sublist) for sublist in rows]

    
    # Convert the rearranged data into a DataFrame
    df = pd.DataFrame(rearranged_data, columns=['Variable Names', 'Most important issue', 'One of the most important  issues'])
    df = df.drop(columns=['Most important issue'])
    df['Date'] = formatted_date_str  # Add a 'Date' column with the formatted date
    df = df.pivot(columns='Variable Names', values='One of the most important  issues', index='Date')

    # Reset the index to make 'Date' a column
    df = df.reset_index()
    df = df.rename_axis('Index', axis=1)

    return df

In [ ]:
def get_issues_data_from_pdf_url(date, url_str):
    global ipsos_issues_df
    # Convert date_str to MM/YYYY format
    date_obj = datetime.strptime(date_str, '%b%y')
    formatted_date_str = date_obj.strftime('%m/%Y')

    # Construct the URL with the given date string
    pdf_url = url_str
    
    # Get the PDF content
    response = requests.get(pdf_url)
    pdf_file = PyPDF2.PdfReader(BytesIO(response.content))
    
   # Extract text from each page of the PDF
    for page_number in range(len(pdf_file.pages)):
        page_text = pdf_file.pages[page_number].extract_text()
    rows = [line.split() for line in page_text.split('\n') if line.strip()]
    rows = rows[10:]
    
    # Rearrange each sublist
    def rearrange_sublist(sublist):
        variable_names = ' '.join(sublist[:-2])
        first_value = sublist[-2] if len(sublist) >= 2 else ''
        second_value = sublist[-1] if len(sublist) >= 1 else ''
        return [variable_names, first_value, second_value]

    rearranged_data = [rearrange_sublist(sublist) for sublist in rows]

    # Convert the rearranged data into a DataFrame
    df = pd.DataFrame(rearranged_data, columns=['Variable Names', 'Most important issue', 'One of the most important  issues'])
    df = df.drop(columns='Most important issue')
    df = df.rename(columns={'One of the most important  issues': date})
    df = df.set_index('Variable Names').T
    df = df.rename_axis('Date', axis=1)
    
    ipsos_issues_df = ipsos_issues_df.append(df)
    
    return ipsos_issues_df

In [1301]:
# Define a list of tuples containing pairs of dates from Jan 2020 to Jan 2024

date_pairs_2019 = [('Jan19', 'Jan19'), ('Feb19', 'Jan19'), ('Feb19', 'Feb19'),
    ('Mar19', 'Feb19'), ('Mar19', 'Mar19'), ('Apr19', 'Mar19'), ('Apr19', 'Apr19'), ('May19', 'Apr19'),
    ('May19', 'May19'), ('Jun19', 'May19'), ('Jun19', 'Jun19'), ('Jul19', 'Jun19'), ('Jul19', 'Jul19'),
    ('Aug19', 'Jul19'), ('Aug19', 'Aug19'), ('Sep19', 'Aug19'), ('Sep19', 'Sep19'), ('Oct19', 'Sep19'),
    ('Oct19', 'Oct19'), ('Nov19', 'Oct19'), ('Nov19', 'Nov19'), ('Dec19', 'Nov19'), ('Dec19', 'Dec19'),
    ('Jan20', 'Dec19')]

date_pairs_2020 = [
    ('Jan20', 'Jan20'), ('Feb20', 'Jan20'), ('Feb20', 'Feb20'), ('Mar20', 'Feb20'), ('Mar20', 'Mar20'),
    ('Apr20', 'Mar20'), ('Apr20', 'Apr20'), ('May20', 'Apr20'), ('May20', 'May20'), ('Jun20', 'May20'),
    ('Jun20', 'Jun20'), ('Jul20', 'Jun20'), ('Jul20', 'Jul20'), ('Aug20', 'Jul20'), ('Aug20', 'Aug20'),
    ('Sep20', 'Aug20'), ('Sep20', 'Sep20'), ('Oct20', 'Sep20'), ('Oct20', 'Oct20'), ('Nov20', 'Oct20'),
    ('Nov20', 'Nov20'), ('Dec20', 'Nov20'), ('Dec20', 'Dec20'), ('Jan21', 'Dec20')]

date_pairs_2021 = [('Jan21', 'Jan21'),
    ('Feb21', 'Jan21'), ('Feb21', 'Feb21'), ('Mar21', 'Feb21'), ('Mar21', 'Mar21'), ('Apr21', 'Mar21'),
    ('Apr21', 'Apr21'), ('May21', 'Apr21'), ('May21', 'May21'), ('Jun21', 'May21'), ('Jun21', 'Jun21'),
    ('Jul21', 'Jun21'), ('Jul21', 'Jul21'), ('Aug21', 'Jul21'), ('Aug21', 'Aug21'), ('Sep21', 'Aug21'),
    ('Sep21', 'Sep21'), ('Oct21', 'Sep21'), ('Oct21', 'Oct21'), ('Nov21', 'Oct21'), ('Nov21', 'Nov21'),
    ('Dec21', 'Nov21'), ('Dec21', 'Dec21'), ('Jan22', 'Dec21')]

date_pairs_2022 = [('Jan22', 'Jan22'), ('Feb22', 'Jan22'),
    ('Feb22', 'Feb22'), ('Mar22', 'Feb22'), ('Mar22', 'Mar22'), ('Apr22', 'Mar22'), ('Apr22', 'Apr22'),
    ('May22', 'Apr22'), ('May22', 'May22'), ('Jun22', 'May22'), ('Jun22', 'Jun22'), ('Jul22', 'Jun22'),
    ('Jul22', 'Jul22'), ('Aug22', 'Jul22'), ('Aug22', 'Aug22'), ('Sep22', 'Aug22'), ('Sep22', 'Sep22'),
    ('Oct22', 'Sep22'), ('Oct22', 'Oct22'), ('Nov22', 'Oct22'), ('Nov22', 'Nov22'), ('Dec22', 'Nov22'),
    ('Dec22', 'Dec22'), ('Jan23', 'Dec22')]

date_pairs_2023 = [('Jan23', 'Jan23'), ('Feb23', 'Jan23'), ('Feb23', 'Feb23'),
    ('Mar23', 'Feb23'), ('Mar23', 'Mar23'), ('Apr23', 'Mar23'), ('Apr23', 'Apr23'), ('May23', 'Apr23'),
    ('May23', 'May23'), ('Jun23', 'May23'), ('Jun23', 'Jun23'), ('Jul23', 'Jun23'), ('Jul23', 'Jul23'),
    ('Aug23', 'Jul23'), ('Aug23', 'Aug23'), ('Sep23', 'Aug23'), ('Sep23', 'Sep23'), ('Oct23', 'Sep23'),
    ('Oct23', 'Oct23'), ('Nov23', 'Oct23'), ('Nov23', 'Nov23'), ('Dec23', 'Nov23'), ('Dec23', 'Dec23'),
    ('Jan24', 'Dec23')]

In [1302]:
ipsos_issues_combined_df_2019 = pd.DataFrame()

# Loop through the dates list and process each date
for date_of_publish_str, date_str in date_pairs_2019:
    try:
        # Call the function to get brexit data for the current date
        ipsos_issues_df_2019 = get_issues_data_from_pdf(date_of_publish_str, date_str)
        
        # Append the current date's data to the combined DataFrame
        ipsos_issues_combined_df_2019 = ipsos_issues_combined_df_2019.append(ipsos_issues_df_2019, ignore_index= False)
    except Exception as e:
        print(f"Error occurred for date: {date_str}")
        print(f"Error message: {e}")
        continue

404 https://www.ipsos.com/sites/default/files/ct/news/documents/2019-01/January19_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2019-02/January19_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2019-02/February19_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2019-04/March19_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2019-04/April19_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2019-06/May19_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2019-06/June19_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2019-07/July19_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2019-08/July19_issues_index_topline_p

In [1296]:
ipsos_issues_combined_df_2020 = pd.DataFrame()

#Loop through the dates list and process each date

for date_of_publish_str, date_str in date_pairs_2020:
    try:
        # Call the function to get brexit data for the current date
        ipsos_issues_df_2020 = get_issues_data_from_pdf(date_of_publish_str, date_str)
        
        # Append the current date's data to the combined DataFrame
        ipsos_issues_combined_df_2020 = ipsos_issues_combined_df_2020.append(ipsos_issues_df_2020, ignore_index= False)
    except Exception as e:
        print(f"Error occurred for date: {date_str}")
        print(f"Error message: {e}")
        continue

404 https://www.ipsos.com/sites/default/files/ct/news/documents/2020-01/January20_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2020-02/February20_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2020-03/February20_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2020-03/March20_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2020-04/March20_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2020-05/April20_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2020-05/May20_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2020-06/May20_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2020-07/June20_issues_index_topline_

In [1299]:
date_pairs_2020_second_half = [('Sep20','Sep20'),('Oct20', 'Sep20'), ('Oct20', 'Oct20'), ('Nov20', 'Oct20'),
    ('Nov20', 'Nov20'), ('Dec20', 'Nov20'), ('Dec20', 'Dec20'), ('Jan21', 'Dec20')]

# Loop through the dates list and process each date
for date_of_publish_str, date_str in date_pairs_2020_second_half:
    try:
        # Call the function to get brexit data for the current date
        ipsos_issues_df_2020 = get_issues_data_from_pdf(date_of_publish_str, date_str)
        
        # Append the current date's data to the combined DataFrame
        ipsos_issues_combined_df_2020 = ipsos_issues_combined_df_2020.append(ipsos_issues_df_2020, ignore_index= True)
    except Exception as e:
        print(f"Error occurred for date: {date_str}")
        print(f"Error message: {e}")
        continue

404 https://www.ipsos.com/sites/default/files/ct/news/documents/2020-10/September20_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2020-10/October20_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2020-11/November20_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2020-12/December20_issues_index_topline_public.pdf


In [1309]:
ipsos_issues_combined_df_2021 = pd.DataFrame()

# Loop through the dates list and process each date
for date_of_publish_str, date_str in date_pairs_2021:
    try:
        # Call the function to get brexit data for the current date
        ipsos_issues_df_2021 = get_issues_data_from_pdf(date_of_publish_str, date_str)
        
        # Append the current date's data to the combined DataFrame
        ipsos_issues_combined_df_2021 = ipsos_issues_combined_df_2021.append(ipsos_issues_df_2021, ignore_index= True)
    except Exception as e:
        print(f"Error occurred for date: {date_str}")
        print(f"Error message: {e}")
        continue

404 https://www.ipsos.com/sites/default/files/ct/news/documents/2021-01/January21_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2021-02/February21_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2021-04/March21_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2021-04/April21_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2021-05/April21_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2021-05/May21_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2021-06/May21_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2021-06/June21_issues_index_topline_public.pdf
404 https://www.ipsos.com/sites/default/files/ct/news/documents/2021-07/July21_issues_index_topline_publ

In [1311]:
date_pairs_2021_second_half = [('Sep21','Sep21'),('Oct21', 'Oct21'), ('Nov21', 'Oct21'),
    ('Nov21', 'Nov21'), ('Dec21', 'Nov21'), ('Dec21', 'Dec21'), ('Jan22', 'Dec21')]

# Loop through the dates list and process each date
for date_of_publish_str, date_str in date_pairs_2021_second_half:
    try:
        # Call the function to get brexit data for the current date
        ipsos_issues_df_2021 = get_issues_data_from_pdf(date_of_publish_str, date_str)
        
        # Append the current date's data to the combined DataFrame
        ipsos_issues_combined_df_2021 = ipsos_issues_combined_df_2021.append(ipsos_issues_df_2021, ignore_index= True)
    except Exception as e:
        print(f"Error occurred for date: {date_str}")
        print(f"Error message: {e}")
        continue

429 https://www.ipsos.com/sites/default/files/ct/news/documents/2020-09/September20_issues_index_topline_public.pdf
429 https://www.ipsos.com/sites/default/files/ct/news/documents/2020-10/September20_issues_index_topline_public.pdf


KeyboardInterrupt: 

In [ ]:
ipsos_issues_combined_df_2022 = pd.DataFrame()

# Loop through the dates list and process each date
for date_of_publish_str, date_str in date_pairs_2022:
    try:
        # Call the function to get brexit data for the current date
        ipsos_issues_df_2022 = get_issues_data_from_pdf(date_of_publish_str, date_str)
        
        # Append the current date's data to the combined DataFrame
        ipsos_issues_combined_df_2022 = ipsos_issues_combined_df_2022.append(ipsos_issues_df_2022, ignore_index= True)
    except Exception as e:
        print(f"Error occurred for date: {date_str}")
        print(f"Error message: {e}")
        continue

In [ ]:
date_pairs_2022_second_half = [('Sep22','Sep22'),('Oct22', 'Oct22'), ('Nov22', 'Oct22'), ('Nov22', 'Nov22'), ('Dec22', 'Nov22'),
    ('Dec22', 'Dec22'), ('Jan23', 'Dec22')]

# Loop through the dates list and process each date
for date_of_publish_str, date_str in date_pairs_2020_second_half:
    try:
        # Call the function to get brexit data for the current date
        ipsos_issues_df_2022 = get_issues_data_from_pdf(date_of_publish_str, date_str)
        
        # Append the current date's data to the combined DataFrame
        ipsos_issues_combined_df_2022 = ipsos_issues_combined_df_2022.append(ipsos_issues_df_2022, ignore_index= True)
    except Exception as e:
        print(f"Error occurred for date: {date_str}")
        print(f"Error message: {e}")
        continue

In [ ]:
ipsos_issues_combined_df_2023 = pd.DataFrame()

# Loop through the dates list and process each date
for date_of_publish_str, date_str in date_pairs_2023:
    try:
        # Call the function to get brexit data for the current date
        ipsos_issues_df_2023 = get_issues_data_from_pdf(date_of_publish_str, date_str)
        
        # Append the current date's data to the combined DataFrame
        ipsos_issues_combined_df_2023 = ipsos_issues_combined_df_2023.append(ipsos_issues_df_2023, ignore_index= True)
    except Exception as e:
        print(f"Error occurred for date: {date_str}")
        print(f"Error message: {e}")
        continue

In [ ]:
date_pairs_2023_second_half = [('Sep23','Sep23'),('Oct23', 'Sep23'),('Oct23', 'Oct23'), ('Nov23', 'Oct23'), ('Nov23', 'Nov23'), ('Dec23', 'Nov23'), ('Dec23', 'Dec23'),
    ('Jan24', 'Dec23')]

# Loop through the dates list and process each date
for date_of_publish_str, date_str in date_pairs_2023_second_half:
    try:
        # Call the function to get brexit data for the current date
        ipsos_issues_df_2023 = get_issues_data_from_pdf(date_of_publish_str, date_str)
        
        # Append the current date's data to the combined DataFrame
        ipsos_issues_combined_df_2023 = ipsos_issues_combined_df_2023.append(ipsos_issues_df_2023, ignore_index= True)
    except Exception as e:
        print(f"Error occurred for date: {date_str}")
        print(f"Error message: {e}")
        continue

In [1273]:
ipsos_issues_df = pd.concat([ipsos_issues_combined_df_2020, ipsos_issues_combined_df_2021, ipsos_issues_combined_df_2022, 
                             ipsos_issues_combined_df_2023, ipsos_issues_df_2024], ignore_index = True)

# Months i couldn't scrape: Apr 21; May 21; Dec 22; Feb23 (all no topline); dec 2023 (weird url); 
# aug 2020, jul2020, may 2020, mar 2020 (all didn't do one), feb 2020 (released 2 months ahead); Jan 2019 and Aug 2019

# receiving data from urls 

# feb 2020: https://www.ipsos.com/sites/default/files/ct/news/documents/2020-04/february20_issues_index_topline_public.pdf

# receiving data from chart pdfs

# formatting (to do)

In [ ]:
#. sort dataframe by date

#ipsos_issues_df.reset_index(inplace=True)
#ipsos_issues_df['index'] = pd.to_datetime(ipsos_issues_df['index'])
#ipsos_issues_df = ipsos_issues_df.drop(columns = 'level_0')
ipsos_issues_df = ipsos_issues_df.sort_values(by='index')
ipsos_issues_df['index'] = ipsos_issues_df['index'].dt.strftime('%m/%Y')
ipsos_issues_df.set_index('index', inplace=True)
#flip so nearest date at top: 

In [ ]:
# code to add new data: either try scraping data or if that doesnt work add by url

# Best PM

In [ ]:
best_pm_link = 'https://wethink.netlify.app/'

link = []

headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
pageTree = requests.get(best_pm_link, headers=headers)
pageSoup = BeautifulSoup(pageTree.content, 'html.parser')
best_pm_url = pageSoup.find_all('div', {'class': 'flex flex-col gap-y-5 items-start'})

dates = []
for time_tag in pageSoup.find_all('time', {'datetime': True}):
    dates.append(time_tag['datetime'])
date = dates[0]


scraped_data = pageSoup.find_all('p', class_='!mb-0')

string_list = [tag.get_text() for tag in scraped_data]

cleaned_data = [s.split(' (')[0] for s in string_list]


keir_starmer = 0
rishi_sunak = 0
dont_know = 0


# Iterate over the data list
for i in range(len(cleaned_data)):
    # Check if the string contains the name and percentage
    if 'Sir Keir Starmer' in cleaned_data[i]:
        keir_starmer = int(cleaned_data[i-1])
    elif 'Rishi Sunak' in cleaned_data[i]:
        rishi_sunak = int(cleaned_data[i-1])
    elif "Don't Know" in cleaned_data[i]:
        dont_know = int(cleaned_data[i-1])
        
best_pm_df_latest = {"Date": date,
        'Keir Starmer': [keir_starmer],
        'Rishi Sunak': [rishi_sunak],
        "Don't know": [dont_know]}

best_pm_df_latest = pd.DataFrame(best_pm_df_latest)
best_pm_df_latest['Date'] = pd.to_datetime(best_pm_df_latest['Date'])
best_pm_df_latest['Date'] = best_pm_df_latest['Date'].dt.strftime('%d-%m-%Y')
best_pm_df_latest.iloc[:, 1:7] /= 100

best_pm_df_historical = pd.read_excel('/Users/james1/Documents/Documents/global counsel/Best_pm_historical_data.xlsx')
best_pm_df_historical['Date'] = best_pm_df_historical['Date'].dt.strftime('%d-%m-%Y')
best_pm_df_final = pd.concat([best_pm_df_latest, best_pm_df_historical], axis=0)

In [ ]:
# code for adding new data

best_pm_link = 'https://wethink.netlify.app/'

link = []

headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
pageTree = requests.get(best_pm_link, headers=headers)
pageSoup = BeautifulSoup(pageTree.content, 'html.parser')
best_pm_url = pageSoup.find_all('div', {'class': 'flex flex-col gap-y-5 items-start'})

dates = []
for time_tag in pageSoup.find_all('time', {'datetime': True}):
    dates.append(time_tag['datetime'])
date = dates[0]


scraped_data = pageSoup.find_all('p', class_='!mb-0')

string_list = [tag.get_text() for tag in scraped_data]

cleaned_data = [s.split(' (')[0] for s in string_list]


keir_starmer = 0
rishi_sunak = 0
dont_know = 0


# Iterate over the data list
for i in range(len(cleaned_data)):
    # Check if the string contains the name and percentage
    if 'Sir Keir Starmer' in cleaned_data[i]:
        keir_starmer = int(cleaned_data[i-1])
    elif 'Rishi Sunak' in cleaned_data[i]:
        rishi_sunak = int(cleaned_data[i-1])
    elif "Don't Know" in cleaned_data[i]:
        dont_know = int(cleaned_data[i-1])
        
best_pm_df_latest = {"Date": date,
        'Keir Starmer': [keir_starmer],
        'Rishi Sunak': [rishi_sunak],
        "Don't know": [dont_know]}

best_pm_df_latest = pd.DataFrame(best_pm_df_latest)
best_pm_df_latest['Date'] = pd.to_datetime(best_pm_df_latest['Date'])
best_pm_df_latest['Date'] = best_pm_df_latest['Date'].dt.strftime('%d-%m-%Y')
best_pm_df_latest.iloc[:, 1:7] /= 100

best_pm_df_final = pd.concat([best_pm_df_latest,best_pm_df_final], axis = 0, ignore_index = True)

# sort dataframe by date

best_pm_df_final['Date'] = pd.to_datetime(best_pm_df_final['Date'],format = '%d-%m-%Y')
best_pm_df_final = best_pm_df_final.sort_values(by='Date')
best_pm_df_final['Date'] = best_pm_df_final['Date'].dt.strftime('%d-%m-%Y')

# get rid of any duplicates 

best_pm_df_final = best_pm_df_final.drop_duplicates(ignore_index = True)

best_pm_df_final.to_excel('bestpm.xlsx', index=False) 

# Economy and Brexit

In [ ]:
# define get brexit and economy data function 

def get_brexit_and_economy_delta_data(date_str):
    # Convert date string to different formats
    formatted_date_YM = date_str[6:] + '/' + date_str[3:5]
    formatted_date_YMD = date_str[8:] + date_str[3:5] + date_str[:2]

    # Construct PDF URL
    pdf_url = f'https://deltapoll.co.uk/wp-content/uploads/{formatted_date_YM}/Deltapoll-{formatted_date_YMD}_trackers.pdf'

    #  # Retrieve PDF content
    response = requests.get(pdf_url)
    pdf_file = PdfReader(BytesIO(response.content))
    page_text = pdf_file.pages[-1].extract_text()

    # Split text into sections
    sections = page_text.split('\n')

    # Trim data
    trimmed_data = []
    for item in sections:
        if 'Excluding' in item:
            break
        trimmed_data.append(item)

    trimmed_data = [item for item in trimmed_data if any(char.isdigit() for char in item)]

    # Initialize lists to hold extracted data
    conservative_number = []
    labour_number = []
    dont_know_economy = []
    stay_out_eu = []
    rejoin_eu = []
    dont_know_brexit = []

    # Extract data
    for item in trimmed_data:
        if 'Exchequer' in item:
            number = ''.join(filter(str.isdigit, item))
            if not conservative_number:
                conservative_number.append(number)
            else:
                labour_number.append(number)
        elif "Don't know" in item:
            number = ''.join(filter(str.isdigit, item))
            if not dont_know_economy:
                dont_know_economy.append(number)
            else:
                dont_know_brexit.append(number)
        elif 'European Union' in item:
            number = ''.join(filter(str.isdigit, item))
            if not stay_out_eu:
                stay_out_eu.append(number)
            else:
                rejoin_eu.append(number)

    # Create data dictionaries
    brexit_economy_data = {
        'Conservatives/Rishi Sunak/Jeremy Hunt': conservative_number,
        'Labour/Keir Starmer/Rachel Reeves': labour_number,
        'Don\'t Know econ': dont_know_economy,
        'Rejoin EU': rejoin_eu,
        'Stay Out EU': stay_out_eu,
        'Don\'t Know brex': dont_know_brexit
    }

    # Create dataframes
    brexit_economy_df = pd.DataFrame(brexit_economy_data)
    brexit_economy_df['Conservatives/Rishi Sunak/Jeremy Hunt'] = pd.to_numeric(brexit_economy_df['Conservatives/Rishi Sunak/Jeremy Hunt'])
    brexit_economy_df['Labour/Keir Starmer/Rachel Reeves'] = pd.to_numeric(brexit_economy_df['Labour/Keir Starmer/Rachel Reeves'])
    brexit_economy_df['Don\'t Know econ'] = pd.to_numeric(brexit_economy_df['Don\'t Know econ'])
    brexit_economy_df['Stay Out EU'] =  pd.to_numeric(brexit_economy_df['Stay Out EU'])
    brexit_economy_df['Rejoin EU'] =  pd.to_numeric(brexit_economy_df['Rejoin EU'])
    brexit_economy_df['Don\'t Know brex'] =  pd.to_numeric(brexit_economy_df['Don\'t Know brex'])

    # Add Date column to dataframes
    date = pd.to_datetime(date_str)
    formatted_date = date.strftime('%d-%m-%Y')
    brexit_economy_df['Date'] = formatted_date
    

    return brexit_economy_df

In [ ]:
# date list for scraping historical data 

dates_list = [
    "19/02/2024","12/02/2024","05/02/2024","29/01/2024","22/01/2024","15/01/2024","11/12/2023","04/12/2023","27/11/2023",
    "20/11/2023","13/11/2023","06/11/2023", "30/10/2023","23/10/2023","16/10/2023", "06/10/2023","03/10/2023","25/09/2023",
    "19/09/2023","04/09/2023","29/08/2023","21/08/2023","14/08/2023","07/08/2023","31/07/2023","24/07/2023","17/07/2023",
    "10/07/2023","03/07/2023","26/06/2023","19/06/2023","12/06/2023","06/06/2023","24/05/2023","15/05/2023","09/05/2023",
    "02/05/2023","27/04/2023","18/04/2023","03/04/2023","27/03/2023","20/03/2023","13/03/2023","07/03/2023","27/02/2023", "21/02/2023"]

In [ ]:
# loop to scrape date list and get historical economy data

brexit_economy_df = pd.DataFrame()

# Loop through the dates list and process each date
for date_str in dates_list:
    try:
        # Call the function to get brexit data for the current date
        data_entry = get_brexit_and_economy_delta_data(date_str)
        
        # Append the current date's data to the combined DataFrame
        brexit_economy_df = brexit_economy_df.append(data_entry, ignore_index=True)
    except Exception as e:
        print(f"Error occurred for date: {date_str}")
        print(f"Error message: {e}")
        continue

In [ ]:
# define function to get brexit and economy from data urls in order to catch any data releases that previous code didn't get


def get_brexit_and_economy_delta_data_url(date_str, url):
   
    # Construct PDF URL
    pdf_url = url
    #  # Retrieve PDF content
    response = requests.get(pdf_url)
    pdf_file = PdfReader(BytesIO(response.content))
    page_text = pdf_file.pages[-1].extract_text()

    # Split text into sections
    sections = page_text.split('\n')

    # Trim data
    trimmed_data = []
    for item in sections:
        if 'Excluding' in item:
            break
        trimmed_data.append(item)

    trimmed_data = [item for item in trimmed_data if any(char.isdigit() for char in item)]

    # Initialize lists to hold extracted data
    conservative_number = []
    labour_number = []
    dont_know_economy = []
    stay_out_eu = []
    rejoin_eu = []
    dont_know_brexit = []

    # Extract data
    for item in trimmed_data:
        if 'Exchequer' in item:
            number = ''.join(filter(str.isdigit, item))
            if not conservative_number:
                conservative_number.append(number)
            else:
                labour_number.append(number)
        elif "Don't know" in item:
            number = ''.join(filter(str.isdigit, item))
            if not dont_know_economy:
                dont_know_economy.append(number)
            else:
                dont_know_brexit.append(number)
        elif 'European Union' in item:
            number = ''.join(filter(str.isdigit, item))
            if not stay_out_eu:
                stay_out_eu.append(number)
            else:
                rejoin_eu.append(number)

    # Create data dictionaries
    brexit_economy_data = {
        'Conservatives/Rishi Sunak/Jeremy Hunt': conservative_number,
        'Labour/Keir Starmer/Rachel Reeves': labour_number,
        'Don\'t Know econ': dont_know_economy,
        'Rejoin EU': rejoin_eu,
        'Stay Out EU': stay_out_eu,
        'Don\'t Know brex': dont_know_brexit
    }

    # Create dataframes
    brexit_economy_df = pd.DataFrame(brexit_economy_data)
    brexit_economy_df['Conservatives/Rishi Sunak/Jeremy Hunt'] = pd.to_numeric(brexit_economy_df['Conservatives/Rishi Sunak/Jeremy Hunt'])
    brexit_economy_df['Labour/Keir Starmer/Rachel Reeves'] = pd.to_numeric(brexit_economy_df['Labour/Keir Starmer/Rachel Reeves'])
    brexit_economy_df['Don\'t Know econ'] = pd.to_numeric(brexit_economy_df['Don\'t Know econ'])
    brexit_economy_df['Stay Out EU'] =  pd.to_numeric(brexit_economy_df['Stay Out EU'])
    brexit_economy_df['Rejoin EU'] =  pd.to_numeric(brexit_economy_df['Rejoin EU'])
    brexit_economy_df['Don\'t Know brex'] =  pd.to_numeric(brexit_economy_df['Don\'t Know brex'])

    # Add Date column to dataframes
    date = pd.to_datetime(date_str)
    formatted_date = date.strftime('%d-%m-%Y')
    brexit_economy_df['Date'] = formatted_date
    

    return brexit_economy_df

In [ ]:
# scrape data using urls: NB this code is only for setting up the historical dataset:see below for adding one new url

date_urls = [
    ("23/10/2023", "https://deltapoll.co.uk/wp-content/uploads/2023/10/Deltapoll-231020_trackers.pdf"),
    ("03/10/2023", 'https://deltapoll.co.uk/wp-content/uploads/2023/10/Deltapoll-231002_trackers.pdf'),
    ("19/09/2023", "https://deltapoll.co.uk/wp-content/uploads/2023/09/Deltapoll-230918_trackers.pdf"),
    ("06/06/2023", "https://deltapoll.co.uk/wp-content/uploads/2023/05/Deltapoll-230606_trackers.pdf"),
    ("24/05/2023", "https://deltapoll.co.uk/wp-content/uploads/2023/05/Deltapoll-230523_trackers.pdf"),
    ("18/04/2023", "https://deltapoll.co.uk/wp-content/uploads/2023/04/Deltapoll-230417_trackers.pdf"),
    ("21/02/2023", "https://deltapoll.co.uk/wp-content/uploads/2023/02/Deltapoll-230220_trackers_1.pdf")
]

brexit_economy_df_url = pd.DataFrame()

for date, url in date_urls:
    try:
        # Call the function to get brexit data for the current date
        brexit_economy_url = get_brexit_and_economy_delta_data_url(date, url)
        # Append the current date's data to the combined DataFrame
        brexit_economy_df_url = brexit_economy_df_url.append(brexit_economy_url, ignore_index=True)
    except Exception as e:
        print(f"Error occurred for date: {date}")
        print(f"Error message: {e}")
        continue
        
brexit_economy_df = pd.concat([brexit_economy_df, brexit_economy_df_url], ignore_index = True)

In [899]:
# sort dataframe by date

brexit_economy_df['Date'] = pd.to_datetime(brexit_economy_df['Date'])
brexit_economy_df = brexit_economy_df.sort_values(by='Date')
brexit_economy_df['Date'] = brexit_economy_df['Date'].dt.strftime('%d-%m-%Y')
#flip so nearest date at top: 

In [ ]:
# code to add one new entry by date

#brexit_economy_entry = get_brexit_and_economy_data_delta(date)
#brexit_economy_df = brexit_economy_df.append(brexit_economy_entry)

# code to add one new entry by url

#brexit_economy_url = get_brexit_and_economy_data_delta_url(date, url)
#brexit_economy_df = brexit_economy_df.append(brexit_economy_url, ignore_index=True)

# Voter satisfaction and economic optimism

In [ ]:
def get_polmon_data_from_pdf(date_of_publish_str, date_str):
    # Define the convert_date_format function
    def convert_date_format(date_of_publish_str):
        # Parse the month and year from the input date string
        month_str = date_of_publish_str[:3]
        year_str = '20' + date_of_publish_str[-2:]

        # Convert the month abbreviation to its numerical representation
        month_dict = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06', 
                      'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}
        month_numeric = month_dict[month_str]

        # Format the date into 'YYYY-MM' format
        formatted_date = f'{year_str}-{month_numeric}'
        return formatted_date
    
    def convert_date_str_to_mmm_yy(date_str): #1: Jan24 -> jan24
        # Parse the month and year from the input date string
        month_str = date_str[:3]
        year_str = date_str[-2:]

        # Convert the month abbreviation to its full name
        month_dict_lower = {'Jan': 'jan', 'Feb': 'feb', 'Mar': 'mar', 'Apr': 'apr', 'May': 'may', 'Jun': 'jun', 
                  'Jul': 'jul', 'Aug': 'aug', 'Sep': 'sep', 'Oct': 'october', 'Nov': 'nov', 'Dec': 'dec'}
        month_lower = month_dict_lower[month_str]

        # Format the date into 'mmm-year' format
        formatted_date_lower = f'{month_lower}{year_str}'
        return formatted_date_lower
    
    def convert_date_str_to_month_yy(date_str): #2 Jan24 -> january24
        # Parse the month and year from the input date string
        month_str = date_str[:3]
        year_str = date_str[-2:]

        # Convert the month abbreviation to its full name
        month_dict_lower = {'Jan': 'january', 'Feb': 'february', 'Mar': 'march', 'Apr': 'april', 'May': 'may', 'Jun': 'june', 
                  'Jul': 'july', 'Aug': 'august', 'Sep': 'september', 'Oct': 'oct', 'Nov': 'november', 'Dec': 'december'}
        month_full_name_lower = month_dict_lower[month_str]

        # Format the date into 'month-year' format
        formatted_date_lower = f'{month_full_name_lower}{year_str}'
        return formatted_date_lower
    
    def convert_date_str_to_Month_yy(date_str): #3 Jan24 -> January24
        # Parse the month and year from the input date string
        month_str = date_str[:3]
        year_str = date_str[-2:]

        # Convert the month abbreviation to its full name
        month_dict_upper = {'Jan': 'January', 'Feb': 'February', 'Mar': 'March', 'Apr': 'April', 'May': 'May', 'Jun': 'June', 
                  'Jul': 'July', 'Aug': 'August', 'Sep': 'September', 'Oct': 'October', 'Nov': 'November', 'Dec': 'December'}
        month_full_name_upper = month_dict_upper[month_str]

        # Format the date into 'Month-Year' format
        formatted_date_upper = f'{month_full_name_upper}{year_str}'
        return formatted_date_upper
    
    def convert_date_str_to_mmm_year(date_str): #4 Jan24 -> jan2024
        # Parse the month and year from the input date string
        month_str = date_str[:3]
        year_str = '20' + date_str[-2:]

        # Convert the month abbreviation to its full name
        month_dict_lower = {'Jan': 'jan', 'Feb': 'feb', 'Mar': 'mar', 'Apr': 'apr', 'May': 'may', 'Jun': 'jun', 
                  'Jul': 'jul', 'Aug': 'aug', 'Sep': 'sep', 'Oct': 'october', 'Nov': 'nov', 'Dec': 'dec'}
        month_lower = month_dict_lower[month_str]

        # Format the date into 'month-year' format
        formatted_date_lower = f'{month_lower}{year_str}'
        return formatted_date_lower
    
    def convert_date_str_to_Mmm_year(date_str): #5 Jan24 -> Jan2024
        # Parse the month and year from the input date string
        month_str = date_str[:3]
        year_str = '20' + date_str[-2:]

        # Convert the month abbreviation to its full name
        month_dict_upper = {'Jan': 'Jan', 'Feb': 'Feb', 'Mar': 'Mar', 'Apr': 'Apr', 'May': 'May', 'Jun': 'Jun', 
                  'Jul': 'Jul', 'Aug': 'Aug', 'Sep': 'Sep', 'Oct': 'Oct', 'Nov': 'Nov', 'Dec': 'Dec'}
        month_upper = month_dict_upper[month_str]

        # Format the date into 'Month-Year' format
        formatted_date_upper = f'{month_upper}{year_str}'
        return formatted_date_upper

    def convert_date_str_to_month_year(date_str): #6 Jan24 -> january2024
        # Parse the month and year from the input date string
        month_str = date_str[:3]
        year_str = '20' + date_str[-2:]

        # Convert the month abbreviation to its full name
        month_dict_lower = {'Jan': 'january', 'Feb': 'february', 'Mar': 'march', 'Apr': 'april', 'May': 'may', 'Jun': 'june', 
                  'Jul': 'july', 'Aug': 'august', 'Sep': 'september', 'Oct': 'october', 'Nov': 'november', 'Dec': 'december'}
        month_full_name_lower = month_dict_lower[month_str]

        # Format the date into 'month-year' format
        formatted_date_lower = f'{month_full_name_lower}{year_str}'
        return formatted_date_lower
        
    def convert_date_str_to_Month_year(date_str): #7 Jan24 -> January2024
        # Parse the month and year from the input date string
        month_str = date_str[:3]
        year_str = '20' + date_str[-2:]

        # Convert the month abbreviation to its full name
        month_dict_upper = {'Jan': 'January', 'Feb': 'February', 'Mar': 'March', 'Apr': 'April', 'May': 'May', 'Jun': 'June', 
                  'Jul': 'July', 'Aug': 'August', 'Sep': 'September', 'Oct': 'October', 'Nov': 'November', 'Dec': 'December'}
        month_full_name_upper = month_dict_upper[month_str]

        # Format the date into 'Month-Year' format
        formatted_date_upper = f'{month_full_name_upper}{year_str}'
        return formatted_date_upper
    
    
    def convert_date_str_to_month_dash_year (date_str): #8 Jan24 -> january-2024
        # Parse the month and year from the input date string
        month_str = date_str[:3]
        year_str = '20' + date_str[-2:]

        # Convert the month abbreviation to its full name
        month_dict_lower = {'Jan': 'january', 'Feb': 'february', 'Mar': 'march', 'Apr': 'april', 'May': 'may', 'Jun': 'june', 
                  'Jul': 'july', 'Aug': 'august', 'Sep': 'september', 'Oct': 'october', 'Nov': 'november', 'Dec': 'december'}
        month_full_name_lower = month_dict_lower[month_str]

        # Format the date into 'month-year' format
        formatted_date_lower = f'{month_full_name_lower}-{year_str}'
        return formatted_date_lower
    
    def convert_date_str_to_mmm_dash_year (date_str): #9 Jan24 -> jan-2024
        # Parse the month and year from the input date string
        month_str = date_str[:3]
        year_str = '20' + date_str[-2:]

        # Convert the month abbreviation to its full name
        month_dict_lower = {'Jan': 'jan', 'Feb': 'feb', 'Mar': 'mar', 'Apr': 'apr', 'May': 'may', 'Jun': 'jun', 
                  'Jul': 'jul', 'Aug': 'aug', 'Sep': 'sep', 'Oct': 'oct', 'Nov': 'nov', 'Dec': 'dec'}
        month_lower = month_dict_lower[month_str]

        # Format the date into 'month-year' format
        formatted_date_lower = f'{month_lower}-{year_str}'
        return formatted_date_lower
    
    # Convert date_of publish_str to 'YYYY-MM' format
    formatted_date_of_publish_str = convert_date_format(date_of_publish_str)
    
    # Convert date_str to MM/YYYY format for the date column of data frame
    date_obj = datetime.strptime(date_str, '%b%y')
    formatted_date_str = date_obj.strftime('%m/%Y')
    
    # Convert date_str into all different formats for scraping 
    date_str_lower_year = convert_date_str_to_mmm_year(date_str) # Jan24 -> jan2024
    date_str_lower_full_month_year = convert_date_str_to_month_year(date_str) #Jan24 -> january2024
    date_str_full_month_year = convert_date_str_to_Month_year(date_str) #Jan24 -> January2024
    date_str_upper_year = convert_date_str_to_Mmm_year(date_str) #Jan24 -> Jan2024
    date_str_lower_yy = convert_date_str_to_mmm_yy(date_str) #Jan24 -> jan24
    date_str_lower_full_month_yy = convert_date_str_to_month_yy(date_str) #Jan24 -> january24
    date_str_full_month_yy = convert_date_str_to_Month_yy(date_str) #Jan24 -> January24
    date_str_lower_full_month_dash_year = convert_date_str_to_month_dash_year(date_str) #Jan24 -> january-2024
    date_str_lower_dash_year = convert_date_str_to_mmm_dash_year(date_str) #Jan24 -> jan-2024

    # List of possible URL formats
    url_formats = [
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/Ipsos%20,
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str}_Issues%20Index_topline_PUBLIC_0.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_lower_year}_Issues%20Index_topline_PUBLIC.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_lower_full_month_year}_Issues%20Index_topline_PUBLIC.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_full_month_year}_Issues%20Index_topline_PUBLIC.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_upper_year}_Issues%20Index_topline_PUBLIC.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_lower_yy}_Issues%20Index_topline_PUBLIC.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_lower_full_month_yy}_Issues%20Index_topline_PUBLIC.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_full_month_yy}_Issues%20Index_topline_PUBLIC.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/ipsos-issues-index-{date_str_lower_full_month_dash_year}-topline.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/ipsos-issues-index-{date_str_lower_full_month_dash_year}-topline-PUBLIC.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/ipsos-issues-index-{date_str_lower_dash_year}-topline.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/ipsos-issues-index-{date_str_lower_dash_year}-topline-PUBLIC.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/ipsos-issues-index-topline-{date_str_lower_dash_year}.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/ipsos-issues-index-topline-{date_str_lower_full_month_dash_year}.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/issues-index-{date_str_lower_full_month_dash_year}-topline.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/issues-index-{date_str_lower_dash_year}-topline.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/issues-index-{date_str_lower_full_month_dash_year}-topline-PUBLIC.pdf',  
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/issues-index-{date_str_lower_dash_year}-topline-PUBLIC.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/issues-index-topline-{date_str_lower_dash_year}.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/issues-index-topline-{date_str_lower_full_month_dash_year}.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str}_issues_index_topline_public.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str}_issues_index_topline.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_lower_year}_issues_index_topline_public.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_lower_full_month_year}_issues_index_topline_public.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_full_month_year}_issues_index_topline_public.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_upper_year}_issues_index_topline_public.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_lower_yy}_issues_index_topline_public.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_lower_full_month_yy}_issues_index_topline_public.pdf',
        f'https://www.ipsos.com/sites/default/files/ct/news/documents/{formatted_date_of_publish_str}/{date_str_full_month_yy}_issues_index_topline_public.pdf'
    ]
        
        https://www.ipsos.com/sites/default/files/ct/news/documents/2023-10/Ipsos%20Oct%202023%20Political%20Monitor%20Charts_251023_PUBLIC_0.pdf
            https://www.ipsos.com/sites/default/files/ct/news/documents/2023-09/Ipsos%20Sept%202023%20Political%20Monitor%20Charts_PUBLIC.pdf
    
    # Attempt to retrieve the PDF content from each URL
    for pdf_url in url_formats:
        response = requests.get(pdf_url)
        if response.status_code == 200:
            pdf_file = PyPDF2.PdfReader(BytesIO(response.content))
            break  # Break out of the loop if PDF content is successfully retrieved
    
    # If PDF content couldn't be retrieved, handle the error or return None
    else:
        print(response.status_code, pdf_url)
        return None
    
    # Get the PDF content
    #response = requests.get(pdf_url)
    #pdf_file = PyPDF2.PdfReader(BytesIO(response.content))
    
     # Extract text from each page of the PDF
    for page_number in range(len(pdf_file.pages)):
        page_text = pdf_file.pages[page_number].extract_text()
    rows = [line.split() for line in page_text.split('\n') if line.strip()]
    rows = rows[10:]
    
    # Rearrange each sublist
    def rearrange_sublist(sublist):
        variable_names = ' '.join(sublist[:-2])
        first_value = sublist[-2] if len(sublist) >= 2 else ''
        second_value = sublist[-1] if len(sublist) >= 1 else ''
        return [variable_names, first_value, second_value]

    rearranged_data = [rearrange_sublist(sublist) for sublist in rows]

    
    # Convert the rearranged data into a DataFrame
    df = pd.DataFrame(rearranged_data, columns=['Variable Names', 'Most important issue', 'One of the most important  issues'])
    df = df.drop(columns=['Most important issue'])
    df['Date'] = formatted_date_str  # Add a 'Date' column with the formatted date
    df = df.pivot(columns='Variable Names', values='One of the most important  issues', index='Date')

    # Reset the index to make 'Date' a column
    df = df.reset_index()
    df = df.rename_axis('Index', axis=1)

    return df

In [1247]:
# code to get historical data

data = {
    "Date": [
        "July 2019", "Sep 2019", "Oct 2019", "Dec 2019",
        "Jan 2020", "March 2020", "June 2020", "July 2020",
        "Sept 2020", "Oct 2020", "Dec 2020", "Jan 2021",
        "March 2021", "April 2021", "May 2021", "July 2021",
        "July 2021", "September 2021", "Oct 2021", "Dec 2021",
        "Jan 2022", "Mar 2022", "Apr 2022", "May 2022",
        "June 2022", "July 2022", "Sept 2022", "Oct 2022",
        "Nov 2022", "Dec 2022", "Jan 2023", "Feb 2023",
        "March 2023", "May 2023", "June 2023", "July 2023",
        "Sept 2023", "Oct 2023", "Nov 2023", "Dec 2023",
        "Jan 2024"],
    
    "Government_Sat": [18, 14, 19, 23, 40, 48, 40, 44, 35, 30, 37, 41, 42, 44, 44, 35, 39, 35, 29, 25,
        25, 31, 30, 26, 23, 20, 20, 11, 16, 14, 14, 15, 16, 15, 12, 14, 12, 14, 13, 12, 13],
    
    "Government_Dis": [75, 81, 74, 72, 50, 41, 51, 48, 57, 61, 53, 51, 49, 51, 48, 55, 51, 51, 62, 65,
        67, 59, 61, 66, 69, 74, 70, 80, 76, 79, 76, 77, 77, 76, 80, 79, 80, 80, 80, 82, 78],

    "Con_Leader_Sat": [31, 37, 46, 36, 47, 52, 48, 47, 40, 33, 42, 42, 44, 44, 44, 38, 41, 39, 34, 28,
        24, 31, 30, 28, 25, 24, 27, 16, 29, 28, 26, 27, 32, 30, 28, 26, 28, 26, 21, 21, 20],
    
    "Con_Leader_Dis": [38, 55, 44, 56, 44, 38, 49, 47, 54, 59, 50, 51, 51, 50, 47, 51, 52, 51, 61, 65,
        70, 59, 63, 64, 69, 69, 29, 67, 49, 56, 59, 54, 59, 55, 63, 63, 66, 69, 66, 69, 66],
    
    "Lab_Leader_Sat": [19, 16, 15, 24, 16, 19, 51, 48, 43, 45, 38, 40, 33, 36, 22, 27, 27, 25, 29, 33,
        27, 33, 32, 31, 32, 29, 31, 31, 36, 36, 37, 34, 31, 30, 31, 31, 31, 30, 30, 30, 30],
    
    "Lab_Leader_Dis": [69, 76, 75, 68, 75, 68, 20, 26, 27, 30, 33, 35, 42, 46, 51, 50, 53, 50, 50, 43,
        42, 43, 46, 49, 48, 49, 50, 45, 42, 40, 40, 46, 51, 54, 50, 49, 48, 48, 51, 51, 48]
}

satisfaction_df = pd.DataFrame(data)
# Subtracting disatisfied from satisfied to get net satisfaction
satisfaction_df['Net_Government'] = satisfaction_df['Government_Sat'] - satisfaction_df['Government_Dis']
satisfaction_df['Net_Con_Leader'] = satisfaction_df['Con_Leader_Sat'] - satisfaction_df['Con_Leader_Dis']
satisfaction_df['Net_Lab_Leader'] = satisfaction_df['Lab_Leader_Sat'] - satisfaction_df['Lab_Leader_Dis']

In [1314]:
# code to update dataframe with new data

pdf_url = 'https://www.ipsos.com/sites/default/files/ct/news/documents/2024-01/ipsos-politicial-monitor-uk-charts-january-2024.pdf'
response = requests.get(pdf_url)
pdf_file = PdfReader(BytesIO(response.content))
page_text = pdf_file.pages[4].extract_text()



PdfReadError: EOF marker not found

# Economic optimism

In [1264]:
data = {
    "Date": [
        "Feb 2019", "March 2019", "May 2019", "June 2019",
        "July 2019", "Sept 2019", "Oct 2019", "Dec 2019",
        "Jan 2020", "March 2020", "June 2020", "July 2020",
        "Sept 2020", "Oct 2020", "Dec 2020", "Jan 2021",
        "March 2021", "April 2021", "May 2021", "July 2021","August 2021",
        "Sept 2021", "Oct 2021", "December 2021", "Jan 2022",
        "March 2022", "Apr 2022", "May 2022", "June 2022", "July 2022",
        "Sept 2022", "Oct 2022", "Nov 2022", "December 2022", "Jan 2023",
        "Feb 2023", "March 2023", "May 2023", "June 2023", "July 2023",
        "Sept 2023", "Oct 2023", "Nov 2023", "Dec 2023", "Jan 2024"
    ],
    "Improve": [
        14, 16, 16, 13, 18, 15, 17, 21, 29, 15, 22, 20, 21, 15, 27, 29, 43, 51, 53,
    47, 44, 31, 28, 25, 27, 14, 14, 15, 13, 16, 15, 16, 16, 14, 24, 23, 22, 24,
    21, 28, 20, 21, 19, 22, 24
    ],
    
    "Stay the same": [ 21, 20, 25, 23, 19, 20, 19, 20, 21, 13, 6, 9, 10, 10, 7, 9, 14, 11, 13, 12,
    12, 13, 17, 19, 13, 7, 12, 10, 8, 10, 10, 11, 10, 14, 16, 12, 17, 18, 18, 15,
    23, 20, 21, 24, 21],
    
    "Get worse": [
        57, 58, 51, 57, 58, 59, 56, 50, 42, 69, 69, 68, 66, 71, 63, 60, 41, 36, 31,
    36, 39, 53, 54, 52, 56, 76, 70, 72, 77, 69, 71, 70, 72, 69, 57, 61, 58, 54,
    58, 55, 55, 55, 55, 50, 50],
    
    "Don't know": [ 9, 6, 9, 7, 5, 6, 8, 9, 7, 3, 3, 2, 3, 4, 4, 2, 3, 2, 4, 5,
    5, 3, 2, 4, 3, 4, 4, 3, 2, 5, 3, 3, 3, 3, 3, 4, 3, 4, 3, 3,
    2, 5, 5, 4, 5],
    
    "EOI": [
        -43, -42, -35, -44, -40, -44, -39, -29, -13, -54, -47, -48, -45, -56,
    -36, -31, 2, 15, 22, 11, 5, -22, -26, -27, -29, -62, -56, -57, -64,
    -53, -56, -54, -56, -55, -33, -38, -36, -30, -37, -27, -35, -34, -36,
    -28, -26]}

econ_optimism_df = pd.DataFrame(data)

In [1266]:
# code for updating every month